In [1]:
import os
import sys

WORKSPACE = os.path.abspath(os.path.pardir)
SRC_DIR = os.path.join(WORKSPACE, "src")
TESTS_DIR = os.path.join(WORKSPACE, "tests")

sys.path.insert(0, TESTS_DIR)
sys.path.insert(0, SRC_DIR)

In [2]:
from enum import Enum

from vcd import VCDWriter

from seqlogic import Module, simify
from seqlogic.bits import T, uint2bits, bits
from seqlogic.sim import Region, get_loop

from riscv.core.singlecycle.top import Top

loop = get_loop()

DEBUG_REG = bits("32hFFFF_FFF0")

In [3]:
def parse_mem(name: str) -> list[int]:
    words = []
    with open(name, encoding="utf-8") as f:
        for line in f:
            for part in line.split()[1:]:
                words.append(int(part, base=16))
    return words


def riscv_build(name: str) -> Top:
    # Create module hierarchy
    top = Top(name="top")

    # Initialize coroutines
    simify(top)

    # Initialize instruction memory
    words = parse_mem(f"../tests/riscv/tests/{name}.text")
    for i, w in enumerate(words):
        top.text_memory_bus.text_memory._mem.set_next(i, uint2bits(w, 32))

    # Initialize data memory
    words = parse_mem(f"../tests/riscv/tests/{name}.data")
    for i, w in enumerate(words):
        top.data_memory_bus.data_memory._mem.set_next(i, uint2bits(w, 32))

    return top


class SimStatus(Enum):
    PASS = 0
    FAIL = 1
    TIMEOUT = 2


def riscv_sim(top: Top) -> SimStatus:
    for _ in loop.iter(until=10000):
        if top.bus_wr_en.value == T and top.bus_addr.value == DEBUG_REG:
            if top.bus_wr_data.value == bits("32h0000_0001"):
                return SimStatus.PASS
            else:
                return SimStatus.FAIL
    return SimStatus.TIMEOUT


def run_test(name: str) -> SimStatus:
    loop.reset()

    with open(f"{name}.vcd", "w") as f:
        with VCDWriter(f, timescale="1ns") as vcdw:
            top = riscv_build(name)
            top.dump_vcd(vcdw, ".*")
            return riscv_sim(top)

In [4]:
assert run_test("add") is SimStatus.PASS

In [5]:
assert run_test("addi") is SimStatus.PASS

In [6]:
assert run_test("and") is SimStatus.PASS

In [7]:
assert run_test("andi") is SimStatus.PASS

In [8]:
assert run_test("auipc") is SimStatus.PASS

In [9]:
assert run_test("beq") is SimStatus.PASS

In [10]:
assert run_test("bge") is SimStatus.PASS

In [11]:
assert run_test("bgeu") is SimStatus.PASS

In [12]:
assert run_test("blt") is SimStatus.PASS

In [13]:
assert run_test("bltu") is SimStatus.PASS

In [14]:
assert run_test("bne") is SimStatus.PASS

In [15]:
assert run_test("jal") is SimStatus.PASS

In [16]:
assert run_test("jalr") is SimStatus.PASS

In [17]:
assert run_test("lb") is SimStatus.PASS

In [18]:
assert run_test("lbu") is SimStatus.PASS

In [19]:
assert run_test("lh") is SimStatus.PASS

In [20]:
assert run_test("lhu") is SimStatus.PASS

In [21]:
assert run_test("lui") is SimStatus.PASS

In [22]:
assert run_test("lw") is SimStatus.PASS

In [23]:
assert run_test("or") is SimStatus.PASS

In [24]:
assert run_test("ori") is SimStatus.PASS

In [25]:
assert run_test("sb") is SimStatus.PASS

In [26]:
assert run_test("sh") is SimStatus.PASS

In [27]:
assert run_test("simple") is SimStatus.PASS

In [28]:
assert run_test("sll") is SimStatus.PASS

In [29]:
assert run_test("slli") is SimStatus.PASS

In [30]:
assert run_test("slt") is SimStatus.PASS

In [31]:
assert run_test("slti") is SimStatus.PASS

In [32]:
assert run_test("sltiu") is SimStatus.PASS

In [33]:
assert run_test("sltu") is SimStatus.PASS

In [34]:
assert run_test("sra") is SimStatus.PASS

In [35]:
assert run_test("srai") is SimStatus.PASS

In [36]:
assert run_test("srl") is SimStatus.PASS

In [37]:
assert run_test("srli") is SimStatus.PASS

In [38]:
assert run_test("sub") is SimStatus.PASS

In [39]:
assert run_test("sw") is SimStatus.PASS

In [40]:
assert run_test("xor") is SimStatus.PASS

In [41]:
assert run_test("xori") is SimStatus.PASS